In [1]:
# Check device number
!ls -ltrh /dev/video*

crw-rw----+ 1 root video 81, 1 Aug 28 16:18 /dev/video1
crw-rw----+ 1 root video 81, 2 Aug 28 16:18 /dev/video2


In [2]:
import torch
from ultralytics import YOLO
import os

print(torch.__version__, "cuda:", torch.cuda.is_available())

#make ft32 tensorrt file
model = YOLO("yolo.pt", task="pose")    # 또는 task="detect"
model.export(format="engine")

os.rename("yolo.engine", "yolo_fp32.engine")
os.rename("yolo.onnx", "yolo_fp32.engine")


2.8.0 cuda: True
WARNING ⚠️ TensorRT requires GPU export, automatically assigning device=0
Ultralytics 8.3.187 🚀 Python-3.10.12 torch-2.8.0 CUDA:0 (Orin, 7620MiB)
YOLO11n-pose summary (fused): 109 layers, 2,956,000 parameters, 0 gradients, 7.8 GFLOPs

PyTorch: starting from 'yolo.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 68, 8400) (6.0 MB)
requirements: Ultralytics requirement ['onnxruntime-gpu'] not found, attempting AutoUpdate...


ERROR: Could not find a version that satisfies the requirement onnxruntime-gpu (from versions: none)
ERROR: No matching distribution found for onnxruntime-gpu


WARNING ⚠️ Retry 1/2 failed: Command 'pip install --no-cache-dir "onnxruntime-gpu" ' returned non-zero exit status 1.


ERROR: Could not find a version that satisfies the requirement onnxruntime-gpu (from versions: none)


WARNING ⚠️ Retry 2/2 failed: Command 'pip install --no-cache-dir "onnxruntime-gpu" ' returned non-zero exit status 1.


ERROR: No matching distribution found for onnxruntime-gpu


WARNING ⚠️ requirements: ❌ Command 'pip install --no-cache-dir "onnxruntime-gpu" ' returned non-zero exit status 1.

ONNX: starting export with onnx 1.12.0 opset 19...
ONNX: slimming with onnxslim 0.1.59...
WARNING ⚠️ ONNX: simplifier failure: module 'onnx.helper' has no attribute 'get_all_tensor_dtypes'
ONNX: export success ✅ 8.4s, saved as 'yolo.onnx' (11.6 MB)

TensorRT: starting export with TensorRT 10.3.0...
[08/28/2025-16:30:27] [TRT] [I] [MemUsageChange] Init CUDA: CPU +1, GPU +0, now: CPU 749, GPU 4510 (MiB)
[08/28/2025-16:30:30] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +928, GPU +1075, now: CPU 1720, GPU 5621 (MiB)
[08/28/2025-16:30:30] [TRT] [I] ----------------------------------------------------------------
[08/28/2025-16:30:30] [TRT] [I] Input filename:   yolo.onnx
[08/28/2025-16:30:30] [TRT] [I] ONNX IR version:  0.0.9
[08/28/2025-16:30:30] [TRT] [I] Opset version:    19
[08/28/2025-16:30:30] [TRT] [I] Producer name:    pytorch
[08/28/2025-16:30:30] [TR

In [7]:
from flask import Flask, Response
from ultralytics import YOLO
import cv2

# YOLO 모델 불러오기 (best.pt or .engine 가능)
model = YOLO("yolo_fp32.engine", task="pose")

c_index = 1 #Camera index can be changed

cap = cv2.VideoCapture(c_index) 
assert cap.isOpened(), "Camera open failed."

app = Flask(__name__)

def gen_frames():
    while True:
        ret, frame = cap.read() 
        if not ret:
            break

        # YOLO inference
        results = model.predict(source=frame, imgsz=640, verbose=False)
        annotated = results[0].plot()  # draw inference results

        # JPEG encoding
        ret, buffer = cv2.imencode('.jpg', annotated)
        if not ret:
            continue #If encoding is failed, skip

        # MJPEG streaming: MJPEG = Motion JPEG
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + buffer.tobytes() + b'\r\n')

@app.route('/')
def video_feed():
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == "__main__":
    try:
        app.run(host="0.0.0.0", port=8000, debug=False, use_reloader=False)
    finally:
        cap.release()
        cv2.destroyAllWindows()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.30.1.50:8000
Press CTRL+C to quit


Loading yolo_fp32.engine for TensorRT inference...
[ERROR ❌ TensorRT model exported with a different version than 10.3.0

08/28/2025-16:38:28] [TRT] [I] Loaded engine size: 11 MiB
[08/28/2025-16:38:28] [TRT] [E] IRuntime::deserializeCudaEngine: Error Code 1: Serialization (Serialization assertion plan->header.magicTag == rt::kPLAN_MAGIC_TAG failed.Trying to load an engine created with incompatible serialization version. Check that the engine was not created using safety runtime, same OS was used and version compatibility parameters were set accordingly.)


172.30.1.93 - - [28/Aug/2025 16:38:28] "GET / HTTP/1.1" 500 -
Error on request:
Traceback (most recent call last):
  File "/home/wonmin/project/hand_keypoints/venv/lib/python3.10/site-packages/werkzeug/serving.py", line 370, in run_wsgi
    execute(self.server.app)
  File "/home/wonmin/project/hand_keypoints/venv/lib/python3.10/site-packages/werkzeug/serving.py", line 333, in execute
    for data in application_iter:
  File "/home/wonmin/project/hand_keypoints/venv/lib/python3.10/site-packages/werkzeug/wsgi.py", line 256, in __next__
    return self._next()
  File "/home/wonmin/project/hand_keypoints/venv/lib/python3.10/site-packages/werkzeug/wrappers/response.py", line 32, in _iter_encoded
    for item in iterable:
  File "/tmp/ipykernel_49439/3885305649.py", line 22, in gen_frames
    results = model.predict(source=frame, imgsz=640, verbose=False)
  File "/home/wonmin/.local/lib/python3.10/site-packages/ultralytics/engine/model.py", line 548, in predict
    self.predictor.setup_model